In [1]:
from ultralytics import YOLO
import cv2
import math
import torch
import numpy as np

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at 'C:\Users\Darshan\AppData\Roaming\Ultralytics\settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'.


In [3]:
GENERIC_WEIGHTS = "D:/RajHack/Rakshak/video-server/models/weights/yolo_v8/yolov8n.pt"
FIRE_WEIGHTS = "D:/RajHack/Rakshak/video-server/models/weights/fire_explosion/fire_optimal.pt"
WEAPONS_WEIGHTS = "D:/RajHack/Rakshak/video-server/models/weights/weapon_detection/weapons.pt"

In [4]:
classes = {
    "generic": ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ],
    "fire" : ["fire"],
    "weapons" : ['Grenade', 'Knife', 'Missile', 'Pistol', 'Rifle']
}

In [5]:
models = {}
models["generic"] = YOLO("yolov8n.pt")
models["fire"] = YOLO(FIRE_WEIGHTS)
models["weapons"] = YOLO(WEAPONS_WEIGHTS)

In [9]:
def process_model(model, img, classNames):
    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # bounding box
            cls = int(box.cls[0])
            accepted_list = ["fire", 'Grenade', 'Knife', 'Pistol', 'Rifle', 'scissors', 'cell phone', 'knife']
            if classNames[cls] not in accepted_list:
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
           

            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

def start_webcam(models, classes):
    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)
    while True:
        success, img = cap.read()

        for model_name, model in models.items():
            process_model(model, img, classes[model_name])
        
        cv2.imshow('Webcam', img)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [10]:
start_webcam(models, classes)


0: 480x640 5 persons, 2 chairs, 107.8ms
Speed: 2.0ms preprocess, 107.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 320x416 (no detections), 56.9ms
Speed: 1.0ms preprocess, 56.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)

0: 480x640 (no detections), 80.4ms
Speed: 2.1ms preprocess, 80.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 1 chair, 107.0ms
Speed: 3.0ms preprocess, 107.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 320x416 (no detections), 55.7ms
Speed: 2.0ms preprocess, 55.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)

0: 480x640 (no detections), 85.9ms
Speed: 3.0ms preprocess, 85.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 2 chairs, 85.1ms
Speed: 1.0ms preprocess, 85.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 320x416 (no detections), 39.9ms
Speed: 1.0ms prepr